# NCI WeatherBench-1b: Create climatology and persistence forecasts (Optional)

In this note book we will create the most basic baselines: persistence and climatology forecasts. We will do this for 500hPa geopotential, 850hPa temperature, precipitation and 2 meter temperature.

## Note: 
**Requires a 382GB ARE instance to load the entire dataset**

# For higher resolutions

Not up to date, but previous tests for Z500 and T850 showed that there was only a tiny difference in the scores for different resolutions.

In [1]:
from datetime import datetime
print( f'[{datetime.now().replace(microsecond=0)}]' )
! export DASK_LOGGING__DISTRIBUTED=error
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from score import *
import logging 
import xarray as xr
import dask
dask.config.set({'logging.distributed': 'error'})
from dask.distributed import Client
import gc
from dask.diagnostics import ProgressBar
import IPython
from dask.distributed import progress
from datetime import datetime

def create_persistence_forecast(ds, lead_time_h):
    assert lead_time_h > 0, 'Lead time must be greater than 0'
    ds_fc = ds.isel(time=slice(0, -lead_time_h))
    return ds_fc

def create_climatology_forecast(ds_train):
    return ds_train.mean('time')

def create_weekly_climatology_forecast(ds_train, valid_time):
    ds_train['week'] = ds_train['time.week']
    weekly_averages = ds_train.groupby('week').mean('time')
    valid_time['week'] = valid_time['time.week']
    fc_list = []
    for t in valid_time:
        fc_list.append(weekly_averages.sel(week=t.week))
    return xr.concat(fc_list, dim=valid_time)

def baseline_forecasts(res): 
    print (80*'-')
    print (f"Res: {res}")
    DATADIR = f'/g/data/wb00/NCI-Weatherbench/{res}deg/' 
    print("DATADIR:", DATADIR)
    PREDDIR = f"/scratch/vp91/{os.environ['USER']}/NCI-Weatherbench/pred_dir" # Location to store baseline forecasts
    print("PREDDIR:", PREDDIR)

    # Set the years data to load
    years       = list(range(1999, 2022+1))
    valid_years = list(range(2021, 2022+1))
    save_prefix = 'NCI_tutorial' 
    print ('load years :', years)
    print ('valid_years:',  valid_years)    
    print ('save_prefix :', save_prefix)
    
    z500_files = [ file for year in years for file in glob.glob (fr'{DATADIR}/geopotential/*{year}*')  ] 
    t850_files = [ file for year in years for file in glob.glob (fr'{DATADIR}/temperature/*{year}*')    ] 
     
    z500_valid_files = [ file for year in valid_years for file in glob.glob (fr'{DATADIR}/geopotential/*{year}*') ] 
    t850_valid_files = [ file for year in valid_years for file in glob.glob (fr'{DATADIR}/temperature/*{year}*')  ] 
           
    print (f'\nLoading data, Res: {res} ...')
    z500 = xr.open_mfdataset(z500_files, combine='by_coords', parallel=True, chunks={'time': 10}).z.sel(level=500).load()  
    t850 = xr.open_mfdataset(t850_files, combine='by_coords', parallel=True, chunks={'time': 10}).t.sel(level=850).load()  

    data = xr.merge([z500.drop('level'), t850.drop('level')])
    print (f'Loading validation data, Res: {res} ...')
    z500_valid = load_test_data(z500_valid_files, 'z', slice('2021', '2022'))
    t850_valid = load_test_data(t850_valid_files,  't', slice('2021', '2022'))   
 
    valid_data = xr.merge([z500_valid, t850_valid])
    
    print("\nPersistence forecast ...")      
    lead_times = xr.DataArray(
    np.arange(6, 126, 6), dims=['lead_time'], coords={'lead_time': np.arange(6, 126, 6)}, name='lead_time')

    persistence = []
    for l in lead_times:
        persistence.append(create_persistence_forecast(valid_data, int(l)))
    persistence = xr.concat(persistence, dim=lead_times)
    
    print ('Saving persistence forecast result:', f'{PREDDIR}/{save_prefix}_persistence_{res}.nc')
    persistence.to_netcdf(   f'{PREDDIR}/{save_prefix}_persistence_{res}.nc')
    print ( (os.path.getsize(f'{PREDDIR}/{save_prefix}_persistence_{res}.nc')/1024**3) )    
    
    print('\nClimatology forecast ...')
    train_data = data.sel(time=slice('1999', '2020'))
    climatology = create_climatology_forecast(train_data)
    print ('Saving climatology forecast result:', f'{PREDDIR}/{save_prefix}_climatology_{res}.nc')
    climatology.to_netcdf(   f'{PREDDIR}/{save_prefix}_climatology_{res}.nc')
    print ( (os.path.getsize(f'{PREDDIR}/{save_prefix}_climatology_{res}.nc')/1024**3) )
 
    print('\nWeekly climatology ...')
    weekly_climatology = create_weekly_climatology_forecast(train_data, valid_data.time)
    print ('Saving weekly climatology result:', f'{PREDDIR}/{save_prefix}_weekly_climatology_{res}.nc')
    weekly_climatology.to_netcdf(f'{PREDDIR}/{save_prefix}_weekly_climatology_{res}.nc')
    print ( (os.path.getsize    (f'{PREDDIR}/{save_prefix}_weekly_climatology_{res}.nc')/1024**3) )
    
    print ("Done")


[2024-03-26 17:39:39]


# '2.8125'

In [2]:
print( f'[{datetime.now().replace(microsecond=0)}]' )
client = Client(n_workers=12, threads_per_worker=1, silence_logs=logging.ERROR)
client

[2024-03-26 17:39:47]


/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.events-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.wa

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 191.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39143,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 191.00 GiB
Comm: tcp://127.0.0.1:34091,Total threads: 1
Dashboard: /proxy/39859/status,Memory: 15.92 GiB
Nanny: tcp://127.0.0.1:45499,


In [3]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
baseline_forecasts('2.8125')  

[2024-03-26 17:39:48]
--------------------------------------------------------------------------------
Res: 2.8125
DATADIR: /g/data/wb00/NCI-Weatherbench/2.8125deg/
PREDDIR: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir
load years : [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
valid_years: [2021, 2022]
save_prefix : NCI_tutorial

Loading data, Res: 2.8125 ...
Loading validation data, Res: 2.8125 ...
load_test_data, var: z
load_test_data, var: t

Persistence forecast ...
Saving persistence forecast result: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial_persistence_2.8125.nc
21.3794065695256

Climatology forecast ...
Saving climatology forecast result: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial_climatology_2.8125.nc
6.98138028383255e-05

Weekly climatology ...
Saving weekly climatology result: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial_weekly

# '1.40625'
Start the Dask cluster

In [4]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
baseline_forecasts('1.40625')

[2024-03-26 18:13:12]
--------------------------------------------------------------------------------
Res: 1.40625
DATADIR: /g/data/wb00/NCI-Weatherbench/1.40625deg/
PREDDIR: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir
load years : [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
valid_years: [2021, 2022]
save_prefix : NCI_tutorial

Loading data, Res: 1.40625 ...
Loading validation data, Res: 1.40625 ...
load_test_data, var: z
load_test_data, var: t

Persistence forecast ...
Saving persistence forecast result: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial_persistence_1.40625.nc
85.51759159378707

Climatology forecast ...
Saving climatology forecast result: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial_climatology_1.40625.nc
0.000254826620221138

Weekly climatology ...
Saving weekly climatology result: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial

# The End